# Data in 3D - Selecting and Functions

We'll continue working with 3 dimensional datasets and continue using `xarray` to apply _functions_ to our data selections.

We'll start by importing the packages we need and loading the [Berkeley Earth](https://berkeleyearth.org/data/) gridded, global surface temperature dataset.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
# load data
fname_in = '../data/Land_and_Ocean_LatLong1.nc'
DS = xr.open_dataset(fname_in,engine='netcdf4')

In [ ]:
# view the dataset
DS

## Selecting a range with `slice`
Let's remind ourselves that climate is a longterm (>30 years) average of conditions on Earth. In the last notebook, we used the `sel` function to look at individual points in time and space. We can also use `sel` to select a range of points to make meaningful subsets of the data.

Let's start by selecting a 30-year chunk of the surface temperature data from 1990-2020. We can select a range by using the `slice` argument on the `time` dimension.

Now we have a new variable, `present` which is the temperature reccord from 1990-2020. It's nice having this data in it's own `DataArray`, but the value of this selection comes from being able to run calculations on the subset.

## Applying built-in functions

Now that we have our subset we can ask questions about the subset of data and use built-in functions to run calculations on that subset. Let's start with a familiar ones, like `mean`, `min`, `max`, `std`.

## Quick linear regression

We can also use built-in functions to calculate trends in the temperature reccord within the subset using `polyfit`.

Let's test the fit data to see what that looks like at a point.